In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
#plotting
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec
from matplotlib.ticker import MaxNLocator


#sklearn
from sklearn.model_selection import cross_val_score,train_test_split

#stats
from scipy import stats
from scipy.stats import skew, boxcox_normmax, norm
from scipy.special import boxcox1p
from statsmodels.stats.outliers_influence import variance_inflation_factor


from sklearn.preprocessing import StandardScaler
pd.options.display.max_columns = 250
pd.options.display.max_rows = 250
plt.style.use('fivethirtyeight')
%matplotlib inline

In [3]:
df=pd.read_csv('/content/train_data.csv')
df1=pd.read_csv('/content/test_data.csv')

In [ ]:
# none_cols=['trip_creation_time','route_schedule_uuid','cutoff_timestamp','od_end_time','destination_center']
# zero_cols=['osrm_distance','osrm_time','factor','segment_factor']
# # for col in zero_cols:
# #     df[col].replace(np.nan, 0, inplace=True)

# for col in none_cols:
#     df[col].replace(np.nan, 'None', inplace=True)

In [4]:
print(df.isna().sum())
print(df1.isna().sum())

UID                                  0
trip_creation_time                1253
route_schedule_uuid               1065
route_type                           0
trip_uuid                            0
source_center                        0
source_name                          0
destination_center                 292
destination_name                     0
od_start_time                        0
od_end_time                        243
start_scan_to_end_scan               0
is_cutoff                            0
cutoff_factor                        0
cutoff_timestamp                   326
actual_distance_to_destination       0
osrm_time                         5077
osrm_distance                     5030
factor                             602
segment_actual_time                  0
segment_osrm_time                    0
segment_osrm_distance                1
segment_factor                     423
actual_time                          1
dtype: int64
UID                               0
trip_creation_t

In [5]:
nullcols = [col for col in df.columns if df[col].isnull().sum() >120000]
df1.drop(nullcols, axis=1, inplace=True)
df.drop(nullcols, axis=1, inplace=True)

In [6]:
df.head()

,UID,trip_creation_time,route_schedule_uuid,route_type,trip_uuid,source_center,source_name,destination_center,destination_name,od_start_time,od_end_time,start_scan_to_end_scan,is_cutoff,cutoff_factor,cutoff_timestamp,actual_distance_to_destination,osrm_time,osrm_distance,factor,segment_actual_time,segment_osrm_time,segment_osrm_distance,segment_factor,actual_time
0,lWYYrbNEdf,2018-09-15 02:23:18.656742,thanos::sroute:9d8ed3f7-a8af-4393-ad7a-c57a720...,FTL,trip-153697819865649878,IND832109AAB,Jamshedpur_Central_I_3 (Jharkhand),IND834002AAB,Ranchi_Hub (Jharkhand),2018-09-15 02:23:18.656742,2018-09-15 06:33:35.824186,713.521495,True,44,2018-09-15 04:45:23,158.807515,778.672716,1228.735753,2.782826,77.988686,20.793472,28.475783,3.292773,100.0
1,iNJcHQlPCm,2018-09-30 15:07:48.971326,thanos::sroute:b9142a73-d68f-46ec-9afc-d145479...,FTL,trip-153832006897106333,IND422011AAD,Nashik_TgrniaRD_I (Maharashtra),IND421302AAG,Bhiwandi_Mankoli_HB (Maharashtra),2018-09-30 22:38:13.903069,2018-10-01 06:48:37.637477,902.759458,True,22,2018-10-01 05:38:30,81.388109,227.479029,NaN,3.625248,68.517914,18.787231,34.612259,4.277350,61.0
2,U6N0M43qwR,2018-09-23 22:15:46.635975,thanos::sroute:883e99fa-50a3-40e0-a2e2-9b12ed6...,FTL,trip-153774094663572416,IND845305AAA,Raxaul_KairiyaT_D (Bihar),NaN,Muzaffrpur_Bbganj_I (Bihar),2018-09-24 03:56:37.016232,2018-09-24 11:15:00.826140,923.666146,True,22,2018-09-24 09:21:30,134.486898,400.072069,1188.943158,4.415969,70.475386,24.371978,29.867950,5.322072,69.0
3,RrJUneuYNj,2018-09-21 05:11:44.833176,thanos::sroute:96a80600-40e1-436b-9161-fa68f9e...,FTL,trip-153750670483274503,IND842001AAA,Muzaffrpur_Bbganj_I (Bihar),IND854326AAB,Purnia_Central_H_2 (Bihar),2018-09-22 10:08:32.593022,2018-09-22 19:17:22.242888,705.451567,True,176,2018-09-22 14:09:17,262.866674,510.123878,999.421258,2.946647,101.876844,19.588977,27.794019,4.265799,300.0
4,7QLdD7iRC9,2018-09-18 01:10:49.936747,thanos::sroute:b43ec86d-59c1-452c-b4a9-807d357...,FTL,trip-153723305402762987,IND411033AAA,Pune_Tathawde_H (Maharashtra),IND501359AAE,Hyderabad_Shamshbd_H (Telangana),2018-09-18 01:10:49.936747,2018-09-18 17:58:48.449251,1109.682268,True,198,2018-09-18 12:14:05,379.389280,NaN,NaN,1.998790,46.571301,22.671495,26.971844,3.553652,286.0


In [7]:
df.shape[1]

24

In [8]:
df1.shape[1]

23

In [9]:
df = df.drop('UID', axis=1)
df1 = df1.drop('UID', axis=1)

In [10]:
df.head()
df.info()
df['od_start_time1']=df['od_start_time']
df['od_end_time1']=df['od_end_time']
df['trip_creation_time1']=df['trip_creation_time']
df1['od_start_time1']=df1['od_start_time']
df1['od_end_time1']=df1['od_end_time']
df1['trip_creation_time1']=df1['trip_creation_time']
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25644 entries, 0 to 25643
Data columns (total 23 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   trip_creation_time              24391 non-null  object 
 1   route_schedule_uuid             24579 non-null  object 
 2   route_type                      25644 non-null  object 
 3   trip_uuid                       25644 non-null  object 
 4   source_center                   25644 non-null  object 
 5   source_name                     25644 non-null  object 
 6   destination_center              25352 non-null  object 
 7   destination_name                25644 non-null  object 
 8   od_start_time                   25644 non-null  object 
 9   od_end_time                     25401 non-null  object 
 10  start_scan_to_end_scan          25644 non-null  float64
 11  is_cutoff                       25644 non-null  bool   
 12  cutoff_factor                   

In [11]:

df['od_start_time'] = pd.to_datetime(df['od_start_time'])
df['od_end_time'] = pd.to_datetime(df['od_end_time'])
df['segment_key'] = df['trip_uuid'] + df['source_center'] + df['destination_center']

segment_cols = ['segment_actual_time', 'segment_osrm_distance', 'segment_osrm_time']

for col in segment_cols:
    df[col + '_sum'] = df.groupby('segment_key')[col].cumsum()

df[[col + '_sum' for col in segment_cols]]
df.info()
df1['od_start_time'] = pd.to_datetime(df1['od_start_time'])
df1['od_end_time'] = pd.to_datetime(df1['od_end_time'])
df1['segment_key'] = df1['trip_uuid'] + df1['source_center'] + df1['destination_center']

segment_cols = ['segment_actual_time', 'segment_osrm_distance', 'segment_osrm_time']

for col in segment_cols:
    df1[col + '_sum'] = df1.groupby('segment_key')[col].cumsum()

df1[[col + '_sum' for col in segment_cols]]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25644 entries, 0 to 25643
Data columns (total 30 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   trip_creation_time              24391 non-null  object        
 1   route_schedule_uuid             24579 non-null  object        
 2   route_type                      25644 non-null  object        
 3   trip_uuid                       25644 non-null  object        
 4   source_center                   25644 non-null  object        
 5   source_name                     25644 non-null  object        
 6   destination_center              25352 non-null  object        
 7   destination_name                25644 non-null  object        
 8   od_start_time                   25644 non-null  datetime64[ns]
 9   od_end_time                     25401 non-null  datetime64[ns]
 10  start_scan_to_end_scan          25644 non-null  float64       
 11  is

,segment_actual_time_sum,segment_osrm_distance_sum,segment_osrm_time_sum
0,16.0,10.8152,7.0
1,22.0,14.7305,12.0
2,15.0,12.1171,11.0
3,25.0,23.4819,21.0
4,38.0,26.8622,24.0
...,...,...,...
19453,36.0,23.8566,16.0
19454,60.0,48.3346,33.0
19455,89.0,73.5058,50.0
19456,107.0,79.7354,54.0


In [12]:
df['od_time_diff_hour'] = (df['od_end_time'] - df['od_start_time']).dt.total_seconds() /(60)
df['od_time_diff_hour']
df1['od_time_diff_hour'] = (df1['od_end_time'] - df1['od_start_time']).dt.total_seconds() /(60)
df1['od_time_diff_hour']
df.head()

,trip_creation_time,route_schedule_uuid,route_type,trip_uuid,source_center,source_name,destination_center,destination_name,od_start_time,od_end_time,start_scan_to_end_scan,is_cutoff,cutoff_factor,cutoff_timestamp,actual_distance_to_destination,osrm_time,osrm_distance,factor,segment_actual_time,segment_osrm_time,segment_osrm_distance,segment_factor,actual_time,od_start_time1,od_end_time1,trip_creation_time1,segment_key,segment_actual_time_sum,segment_osrm_distance_sum,segment_osrm_time_sum,od_time_diff_hour
0,2018-09-15 02:23:18.656742,thanos::sroute:9d8ed3f7-a8af-4393-ad7a-c57a720...,FTL,trip-153697819865649878,IND832109AAB,Jamshedpur_Central_I_3 (Jharkhand),IND834002AAB,Ranchi_Hub (Jharkhand),2018-09-15 02:23:18.656742,2018-09-15 06:33:35.824186,713.521495,True,44,2018-09-15 04:45:23,158.807515,778.672716,1228.735753,2.782826,77.988686,20.793472,28.475783,3.292773,100.0,2018-09-15 02:23:18.656742,2018-09-15 06:33:35.824186,2018-09-15 02:23:18.656742,trip-153697819865649878IND832109AABIND834002AAB,77.988686,28.475783,20.793472,250.286124
1,2018-09-30 15:07:48.971326,thanos::sroute:b9142a73-d68f-46ec-9afc-d145479...,FTL,trip-153832006897106333,IND422011AAD,Nashik_TgrniaRD_I (Maharashtra),IND421302AAG,Bhiwandi_Mankoli_HB (Maharashtra),2018-09-30 22:38:13.903069,2018-10-01 06:48:37.637477,902.759458,True,22,2018-10-01 05:38:30,81.388109,227.479029,NaN,3.625248,68.517914,18.787231,34.612259,4.277350,61.0,2018-09-30 22:38:13.903069,2018-10-01 06:48:37.637477,2018-09-30 15:07:48.971326,trip-153832006897106333IND422011AADIND421302AAG,68.517914,34.612259,18.787231,490.395573
2,2018-09-23 22:15:46.635975,thanos::sroute:883e99fa-50a3-40e0-a2e2-9b12ed6...,FTL,trip-153774094663572416,IND845305AAA,Raxaul_KairiyaT_D (Bihar),NaN,Muzaffrpur_Bbganj_I (Bihar),2018-09-24 03:56:37.016232,2018-09-24 11:15:00.826140,923.666146,True,22,2018-09-24 09:21:30,134.486898,400.072069,1188.943158,4.415969,70.475386,24.371978,29.867950,5.322072,69.0,2018-09-24 03:56:37.016232,2018-09-24 11:15:00.826140,2018-09-23 22:15:46.635975,NaN,NaN,NaN,NaN,438.396832
3,2018-09-21 05:11:44.833176,thanos::sroute:96a80600-40e1-436b-9161-fa68f9e...,FTL,trip-153750670483274503,IND842001AAA,Muzaffrpur_Bbganj_I (Bihar),IND854326AAB,Purnia_Central_H_2 (Bihar),2018-09-22 10:08:32.593022,2018-09-22 19:17:22.242888,705.451567,True,176,2018-09-22 14:09:17,262.866674,510.123878,999.421258,2.946647,101.876844,19.588977,27.794019,4.265799,300.0,2018-09-22 10:08:32.593022,2018-09-22 19:17:22.242888,2018-09-21 05:11:44.833176,trip-153750670483274503IND842001AAAIND854326AAB,101.876844,27.794019,19.588977,548.827498
4,2018-09-18 01:10:49.936747,thanos::sroute:b43ec86d-59c1-452c-b4a9-807d357...,FTL,trip-153723305402762987,IND411033AAA,Pune_Tathawde_H (Maharashtra),IND501359AAE,Hyderabad_Shamshbd_H (Telangana),2018-09-18 01:10:49.936747,2018-09-18 17:58:48.449251,1109.682268,True,198,2018-09-18 12:14:05,379.389280,NaN,NaN,1.998790,46.571301,22.671495,26.971844,3.553652,286.0,2018-09-18 01:10:49.936747,2018-09-18 17:58:48.449251,2018-09-18 01:10:49.936747,trip-153723305402762987IND411033AAAIND501359AAE,46.571301,26.971844,22.671495,1007.975208


In [18]:
def place2state(x):
    if isinstance(x, str) and '(' in x:
        state = x.split('(')[1]
        return state[:-1]  # remove the closing ')'
    else:
        return None


def place2city(x):
    if isinstance(x, str) and ' (' in x:
        city = x.split(' (')[0]
        city = city.split('_')[0]
        return city
    else:
        return None


def place2city_place(x):
    if isinstance(x, str) and ' (' in x:
        city = x.split(' (')[0]
        return city
    else:
        return None

def place2code(x):
    if isinstance(x, str) and ' (' in x:
        city = x.split(' (')[0]
        code = city.split('_')[-1]
        return code
    else:
        return None


In [19]:
df1['destination_state'] = df1['destination_name'].apply(lambda x: place2state(x))
df['destination_city']  = df['destination_name'].apply(lambda x: place2city(x))
df['destination_place'] = df['destination_name'].apply(lambda x: place2city_place(x))
df['destination_code']  = df['destination_name'].apply(lambda x: place2code(x))
df1['destination_state'] = df1['destination_name'].apply(lambda x: place2state(x))
df1['destination_city']  = df1['destination_name'].apply(lambda x: place2city(x))
df1['destination_place'] = df1['destination_name'].apply(lambda x: place2city_place(x))
df1['destination_code']  = df1['destination_name'].apply(lambda x: place2code(x))

In [20]:
df['trip_creation_time'] = pd.to_datetime(df['trip_creation_time'])

df['trip_year'] = df['trip_creation_time'].dt.year
df['trip_month'] = df['trip_creation_time'].dt.month
df['trip_hour'] = df['trip_creation_time'].dt.hour
df['trip_day'] = df['trip_creation_time'].dt.day
df['trip_week'] = df['trip_creation_time'].dt.isocalendar().week
df['trip_dayofweek'] = df['trip_creation_time'].dt.dayofweek
df1['trip_creation_time'] = pd.to_datetime(df1['trip_creation_time'])

df1['trip_year'] = df1['trip_creation_time'].dt.year
df1['trip_month'] = df1['trip_creation_time'].dt.month
df1['trip_hour'] = df1['trip_creation_time'].dt.hour
df1['trip_day'] = df1['trip_creation_time'].dt.day
df1['trip_week'] = df1['trip_creation_time'].dt.isocalendar().week
df1['trip_dayofweek'] = df1['trip_creation_time'].dt.dayofweek

In [21]:
num_cols = ['start_scan_to_end_scan','actual_distance_to_destination','segment_actual_time_sum','osrm_time','osrm_distance','segment_osrm_distance_sum',
           'segment_osrm_time_sum', 'od_time_diff_hour']

In [22]:
df['route_type'] = df['route_type'].map({'FTL':0, 'Carting':1})
df1['route_type'] = df1['route_type'].map({'FTL':0, 'Carting':1})

In [23]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df[num_cols])
df[num_cols] = scaler.transform(df[num_cols])
num_cols = ['start_scan_to_end_scan','actual_distance_to_destination','segment_actual_time_sum','osrm_time','osrm_distance','segment_osrm_distance_sum',
           'segment_osrm_time_sum', 'od_time_diff_hour']
scaler = StandardScaler()
scaler.fit(df1[num_cols])
df1[num_cols] = scaler.transform(df1[num_cols])

In [24]:
df.head()
df.info()
df1.info()

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25644 entries, 0 to 25643
Data columns (total 41 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   trip_creation_time              24391 non-null  datetime64[ns]
 1   route_schedule_uuid             24579 non-null  object        
 2   route_type                      25644 non-null  int64         
 3   trip_uuid                       25644 non-null  object        
 4   source_center                   25644 non-null  object        
 5   source_name                     25644 non-null  object        
 6   destination_center              25352 non-null  object        
 7   destination_name                25644 non-null  object        
 8   od_start_time                   25644 non-null  datetime64[ns]
 9   od_end_time                     25401 non-null  datetime64[ns]
 10  start_scan_to_end_scan          25644 non-null  float64       
 11  is

In [26]:
!pip install category_encoders
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 5.8 MB/s eta 0:00:00


In [27]:
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder
# for col in df.columns:
#     if df[col].dtype == 'object' or df[col].dtype == 'bool':
#         le = ce.HashingEncoder(n_components=1)
#         df[col] = le.fit_transform(df[col])

# for col in df1.columns:
#     if df1[col].dtype == 'object' or df1[col].dtype == 'bool':
#         le = ce.HashingEncoder(n_components=1)
#         df1[col] = le.fit_transform(df1[col])
for col in df.columns:
    if df[col].dtype == 'object' or df[col].dtype == 'bool':
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])

for col in df1.columns:
    if df1[col].dtype == 'object' or df1[col].dtype == 'bool':
        le = LabelEncoder()
        df1[col] = le.fit_transform(df1[col])
# encoder_purpose = ce.HashingEncoder(n_components=6)
# encoder_purpose.fit_transform(df[])
# encoder_purpose.fit_transform(df1)

In [28]:
df.head()
df.isnull().sum()
df.info()
df.drop(['trip_week','trip_creation_time','od_start_time','od_end_time'],axis=1,inplace=True)
df1.drop(['trip_week','trip_creation_time','od_start_time','od_end_time'],axis=1,inplace=True)
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25644 entries, 0 to 25643
Data columns (total 41 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   trip_creation_time              24391 non-null  datetime64[ns]
 1   route_schedule_uuid             25644 non-null  int64         
 2   route_type                      25644 non-null  int64         
 3   trip_uuid                       25644 non-null  int64         
 4   source_center                   25644 non-null  int64         
 5   source_name                     25644 non-null  int64         
 6   destination_center              25644 non-null  int64         
 7   destination_name                25644 non-null  int64         
 8   od_start_time                   25644 non-null  datetime64[ns]
 9   od_end_time                     25401 non-null  datetime64[ns]
 10  start_scan_to_end_scan          25644 non-null  float64       
 11  is

In [29]:
df.fillna(df.mean(), inplace=True)

df1.fillna(df1.mean(), inplace=True)

In [30]:
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25644 entries, 0 to 25643
Data columns (total 37 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   route_schedule_uuid             25644 non-null  int64  
 1   route_type                      25644 non-null  int64  
 2   trip_uuid                       25644 non-null  int64  
 3   source_center                   25644 non-null  int64  
 4   source_name                     25644 non-null  int64  
 5   destination_center              25644 non-null  int64  
 6   destination_name                25644 non-null  int64  
 7   start_scan_to_end_scan          25644 non-null  float64
 8   is_cutoff                       25644 non-null  int64  
 9   cutoff_factor                   25644 non-null  int64  
 10  cutoff_timestamp                25644 non-null  int64  
 11  actual_distance_to_destination  25644 non-null  float64
 12  osrm_time                       

In [31]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit


def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

for col in df.columns:
    replace_with_thresholds(df, col)

for col in df1.columns:
    replace_with_thresholds(df1, col)


/tmp/ipython-input-2395875970.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-1.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
/tmp/ipython-input-2395875970.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-13824.779999999999' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
/tmp/ipython-input-2395875970.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-1941.8549999999996' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dataframe.loc[(dataframe[variable] < low_limi

In [32]:
cor = df.corr()

In [34]:
X = df.drop(["actual_time"], axis=1)  # Feature Matrix
y = df["actual_time"]


In [35]:
names = X.columns
indexes = X.index
sc = StandardScaler()
X = sc.fit_transform(X)
X = pd.DataFrame(X, columns=names, index=indexes)

names = df1.columns
indexes = df1.index
sc = StandardScaler()
df1 = sc.fit_transform(df1)
df1 = pd.DataFrame(df1, columns=names, index=indexes)

In [36]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.001, random_state=42)
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import mean_squared_error, accuracy_score, classification_report, confusion_matrix

In [38]:
!pip install catboost

from catboost import CatBoostRegressor



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.7 MB/s eta 0:00:00


In [39]:
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from lightgbm import LGBMRegressor
nn_model=CatBoostRegressor(n_estimators=20000,learning_rate=0.075,max_depth=9)
nn_model.fit(X_train,y_train)

Streaming output truncated to the last 5000 lines.
15000:	learn: 0.0522646	total: 15m 11s	remaining: 5m 3s
15001:	learn: 0.0522450	total: 15m 11s	remaining: 5m 3s
15002:	learn: 0.0522145	total: 15m 11s	remaining: 5m 3s
15003:	learn: 0.0521983	total: 15m 11s	remaining: 5m 3s
15004:	learn: 0.0521752	total: 15m 11s	remaining: 5m 3s
15005:	learn: 0.0521633	total: 15m 11s	remaining: 5m 3s
15006:	learn: 0.0521491	total: 15m 12s	remaining: 5m 3s
15007:	learn: 0.0521347	total: 15m 12s	remaining: 5m 3s
15008:	learn: 0.0521177	total: 15m 12s	remaining: 5m 3s
15009:	learn: 0.0521039	total: 15m 12s	remaining: 5m 3s
15010:	learn: 0.0520842	total: 15m 12s	remaining: 5m 3s
15011:	learn: 0.0520637	total: 15m 12s	remaining: 5m 3s
15012:	learn: 0.0520484	total: 15m 12s	remaining: 5m 3s
15013:	learn: 0.0520350	total: 15m 12s	remaining: 5m 3s
15014:	learn: 0.0520250	total: 15m 12s	remaining: 5m 3s
15015:	learn: 0.0520091	total: 15m 12s	remaining: 5m 3s
15016:	learn: 0.0519931	total: 15m 12s	remaining: 5m 

In [40]:
y_val_pred = nn_model.predict(X_val)

In [41]:
X_test = df1
X_test = X_test[X_train.columns]

In [42]:
rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
print(f"Root Mean Squared Error: {rmse}")

Root Mean Squared Error: 26.621100619664674


In [43]:
y_pred = nn_model.predict(X_test)
X_test.info()
y_pred

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19458 entries, 0 to 19457
Data columns (total 36 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   route_schedule_uuid             19458 non-null  float64
 1   route_type                      19458 non-null  float64
 2   trip_uuid                       19458 non-null  float64
 3   source_center                   19458 non-null  float64
 4   source_name                     19458 non-null  float64
 5   destination_center              19458 non-null  float64
 6   destination_name                19458 non-null  float64
 7   start_scan_to_end_scan          19458 non-null  float64
 8   is_cutoff                       19458 non-null  float64
 9   cutoff_factor                   19458 non-null  float64
 10  cutoff_timestamp                19458 non-null  float64
 11  actual_distance_to_destination  19458 non-null  float64
 12  osrm_time                       

array([ 46.17851825,  69.59847566,  13.68061528, ..., 362.75888485,
       386.36755042, 464.92235394])

In [51]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Predict on validation data
y_pred = nn_model.predict(X_val)

# Evaluate
mae = mean_absolute_error(y_val, y_pred)
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_val, y_pred)

print("🔹 MAE:", round(mae, 2))
print("🔹 MSE:", round(mse, 2))
print("🔹 RMSE:", round(rmse, 2))
print("🔹 R² Score:", round(r2, 4))


🔹 MAE: 18.26
🔹 MSE: 708.68
🔹 RMSE: 26.62
🔹 R² Score: 0.9985


In [53]:


# 🔹 Convert to DataFrame
comparison_df = pd.DataFrame({
    'Actual': y_val.values.flatten(),
    'Predicted': y_pred.flatten()
})

# 🔹 Show first 10 rows
print(comparison_df.head(50))


    Actual    Predicted
0     33.0    38.647036
1     64.0    77.233610
2    237.0   196.544770
3    222.0   222.034179
4     33.0    32.532168
5     48.0    60.670656
6    502.0   510.928352
7   1300.0  1211.201670
8    118.0   110.688781
9     48.0    29.305496
10    93.0    84.315884
11    48.0    49.825278
12  2563.0  2542.413960
13   139.0   182.464954
14   111.0    79.069620
15   726.0   742.155242
16    42.0    65.353179
17    60.0    78.324834
18    44.0    34.574259
19    56.0    56.587091
20   254.0   264.359935
21    52.0    49.849795
22    73.0    74.193585
23    53.0    65.092148
24  1619.0  1586.132066
25  2144.0  2098.491527
